In [1]:
import pandas as pd
import datetime

In [2]:
pwd

u'/Users/aurelperianu/Documents/Medicare_payments_classifier'

In [3]:
#data_all=pd.read_csv('Medicare_Data/2012MedicareProviderPayment.txt', delimiter='\t')
provider_all=pd.read_csv('Medicare_Data/Physician_Compare_2014/National_Downloadable_File.csv')

/Users/aurelperianu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,25,26,28,29,30,31,32,33,34,36,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#fct rename columns lower letters; remove spaces
def lower_columns(pdx):
    cols = pdx.columns.tolist()
    cols = [col.lower() for col in cols]
    pdx.columns = cols
    pdx.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
    return pdx
provider_all=lower_columns(provider_all)

In [5]:
#add zip code
provider_all['zip_code']=provider_all['zip_code'].astype(str)
provider_all['zip_base']=provider_all['zip_code'].str.slice(0,5)

#data_all['nppes_provider_zip_base'].unique()

In [6]:
#provider_all['professional_accepts_medicare_assignment']=provider_all['professional_accepts_medicare_assignment'].astype(str)
#provider_all['state']=provider_all['state'].astype(str)

In [7]:
#select WA state providers
#provider_wa['professional_accepts_medicare_assignment'=='M']
#provider_wa=provider_all[provider_all['state']=='WA']
#provider_wa.info()

In [8]:
#provider_all['professional_accepts_medicare_assignment'].unique()

In [9]:
#provider_all.head().T

In [10]:
#drop columns
drop_col=['professional_accepts_medicare_assignment','marker_of_address_line_2_suppression',\
         'participated_in_the_medicare_maintenance_of_certification_program.']
def drop_columns(col_remove,pdx):
    columns=list(pdx.columns.values)
    for col in col_remove:
        if col in columns:
            pdx.drop(col, axis=1, inplace=True)
    return pdx

In [11]:
provider_all=drop_columns(drop_col, provider_all)

In [12]:
provider_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247118 entries, 0 to 2247117
Data columns (total 40 columns):
npi                                                                  int64
pac_id                                                               int64
professional_enrollment_id                                           object
last_name                                                            object
first_name                                                           object
middle_name                                                          object
suffix                                                               object
gender                                                               object
credential                                                           object
medical_school_name                                                  object
graduation_year                                                      float64
primary_specialty                                   

In [13]:
#change column names
#col_to_rename={'participated_in_the_medicare_maintenance_of_certification_program.':'medicare_certified'\
#               ,'committed_to_heart_health_through_the_million_hearts®_initiative.':'hearth_initiative'}
#provider_all.rename(columns=col_to_rename, inplace=True)

In [14]:
#rename columns
#fct remane columns
def name_columns(value_old,value_new,pdx):
    cols = pdx.columns.tolist()
    cols = [col.replace(value_old,value_new) for col in cols]
    pdx.columns = cols
    return pdx
provider_all=name_columns('professional_enrollment_id', 'prof_id', provider_all)
provider_all=name_columns('hospital_affiliation', 'related_org', provider_all)
provider_all=name_columns('secondary_specialty', 'specialty', provider_all)
#provider_all=name_columns('participated_in_the_medicare_maintenance_of_certification_program.', 'medicare_certified', provider_all)
provider_all=name_columns('committed_to_heart_health_through_the_million_hearts®_initiative.', 'hearth_initiative', provider_all)


In [15]:
provider_all=name_columns('number_of_group_practice_members', 'practice_size', provider_all)
provider_all=name_columns('all_secondary_specialties', 'secondary_spec', provider_all)
provider_all=name_columns('reported_quality_measures', 'quality_measures', provider_all)
provider_all=name_columns('used_electronic_health_records', 'electronic_measures', provider_all)
provider_all=name_columns('organization_legal_name', 'org_name', provider_all)
provider_all=name_columns('line_1_street_address', 'address', provider_all)
provider_all=name_columns('line_2_street_address', 'address_extra', provider_all)
provider_all=name_columns('group_practice_pac_id', 'group_pac_id', provider_all)
provider_all=name_columns('medical_school_name', 'medical_school', provider_all)


In [16]:
#str_cols = provider_all.columns[provider_all.dtypes==object]
#provider_all[str_cols] = provider_all[str_cols].fillna('.')


#provider_all.fillna(0,inplace=True)

In [17]:
#convert columns to numeric
#provider_all=provider_all.apply(lambda x: pd.to_numeric(x, errors='ignore'))

numeric=['npi','pac_id','graduation_year','group_pac_id','practice_size','related_org_ccn_1','related_org_ccn_2','related_org_ccn_3',\
        'related_org_ccn_4','related_org_ccn_5','zip_base']

provider_all[numeric] = provider_all[numeric].apply(pd.to_numeric, errors='coerce')

#pd.to_numeric(s, errors='coerce')

In [18]:
#fill all NAN
provider_all=provider_all.replace(['nan','NaN'], ['.','0'])
provider_all.fillna(0,inplace=True)

#convert columns to string
# non_numeric=['prof_id','last_name','first_name','middle_name','suffix','gender','credential','medical_school','primary_specialty',\
#              'specialty_1','specialty_2','specialty_3','specialty_4','secondary_spec','org_name','address','address_extra','city',\
#              'state','zip_code','phone_number','related_org_lbn_1','related_org_lbn_2','related_org_lbn_3','related_org_lbn_4',\
#              'related_org_lbn_5','quality_measures','electronic_measures','hearth_initiative']
non_numeric= provider_all.columns[provider_all.dtypes==object]
provider_all[non_numeric] = provider_all[non_numeric].astype(str)
provider_all['group_pac_id'] = provider_all['group_pac_id'].astype(int)
#provider_all=provider_all.replace('nan', '.')

In [19]:
#provider_all.info()

In [20]:
#provider_all.head(200).T

In [21]:
#set an index
provider_all=provider_all.set_index(provider_all.npi)

In [22]:
#graduation year-convert to years of experience

def f(row):
    now = datetime.datetime.now()
    curr_year=now.year
    if row == 0:
        val = 0
    else: 
        val=curr_year-row
    return val
provider_all['years_experience'] = provider_all['graduation_year'].apply(f)


In [23]:
org_cols=['prof_id',
 'org_name',
 'group_pac_id',
 'practice_size',
 'address',
 'address_extra',
 'city',
 'state',
 'zip_code',
 'phone_number',
 #'related_org_ccn_1',
 'related_org_lbn_1',
 #'related_org_ccn_2',
 'related_org_lbn_2',
 #'related_org_ccn_3',
 'related_org_lbn_3',
 #'related_org_ccn_4',
 'related_org_lbn_4',
 #'related_org_ccn_5',
 'related_org_lbn_5',
 'zip_base']
#drop group columns
org_all=provider_all[org_cols]

#provider_all.info()

# second columns drop for group
# org_cols2=[
#  'prof_id',
#  'address',
#  'address_extra',
#  'city',
#  'zip_code',
#  'phone_number',
#  #'related_org_ccn_1',
#  'related_org_lbn_1',
# #'related_org_ccn_2',
#  'related_org_lbn_2',
#  #'related_org_ccn_3',
#  'related_org_lbn_3',
#  #'related_org_ccn_4',
#  'related_org_lbn_4',
#  #'related_org_ccn_5',
#  'related_org_lbn_5',
#  'zip_base']

#org_all=drop_columns(org_cols2, org_all)



In [24]:
print org_cols

['prof_id', 'org_name', 'group_pac_id', 'practice_size', 'address', 'address_extra', 'city', 'state', 'zip_code', 'phone_number', 'related_org_lbn_1', 'related_org_lbn_2', 'related_org_lbn_3', 'related_org_lbn_4', 'related_org_lbn_5', 'zip_base']


In [25]:
provider_drop = [x for x in org_cols if x != 'state']
# provider_drop=org_cols.remove('state')
#print provider_drop
provider_all=drop_columns(provider_drop, provider_all)
specialty_cols=[
    'specialty_1',
    'specialty_2',
    'specialty_3',
    'specialty_4']
provider_all=drop_columns(specialty_cols, provider_all)

In [26]:
org_all.shape

(2247118, 16)

In [27]:
provider_all.shape

(2247118, 22)

In [28]:
provider_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2247118 entries, 1821199704 to 1023405859
Data columns (total 22 columns):
npi                    int64
pac_id                 int64
last_name              object
first_name             object
middle_name            object
suffix                 object
gender                 object
credential             object
medical_school         object
graduation_year        float64
primary_specialty      object
secondary_spec         object
state                  object
related_org_ccn_1      float64
related_org_ccn_2      float64
related_org_ccn_3      float64
related_org_ccn_4      float64
related_org_ccn_5      float64
quality_measures       object
electronic_measures    object
hearth_initiative      object
years_experience       float64
dtypes: float64(7), int64(2), object(13)
memory usage: 394.3+ MB


In [29]:
#remove duplicates for provider and group files
org_all=org_all.drop_duplicates(subset='group_pac_id', keep='last')

#provider_all=provider_all.drop_duplicates(keep='last')
provider_all=provider_all.drop_duplicates(subset=['npi','last_name'],keep='last')
#org_all['org_name'].value_counts()

In [30]:
org_all.head()

,prof_id,org_name,group_pac_id,practice_size,address,address_extra,city,state,zip_code,phone_number,related_org_lbn_1,related_org_lbn_2,related_org_lbn_3,related_org_lbn_4,related_org_lbn_5,zip_base
npi,,,,,,,,,,,,,,,,
1992864284,I20100625000079,SARACENO CHIROPRACTIC PLLC,8325366313,1.0,83 ROUNDTREE DR,0,PLAINVIEW,NY,118034011,5166221113.0,0,0,0,0,0,11803
1245235357,I20101214000030,"BRIGGSY, P.C.",3173515152,1.0,264 HOGAN BLVD,0,MILL HALL,PA,177511907,5707485099.0,0,0,0,0,0,17751
1942385307,I20120522000836,"DAVID K. TAYLOR, CHIROPRACTOR, INC",3476716176,1.0,16450 -2 S TAMIAMI TRAIL,0,FORT MYERS,FL,339085307,2394333898.0,0,0,0,0,0,33908
1073628392,I20101111001312,MICHAEL KIMMELMAN OD PA,8729271598,1.0,2279 S UNIVERSITY DR,0,DAVIE,FL,333245828,9544730100.0,0,0,0,0,0,33324
1235290263,I20070319000362,HINRICHS CHIROPRACTIC INC.,9638272339,1.0,11304 DAVENPORT ST,0,OMAHA,NE,681542630,4029345830.0,0,0,0,0,0,68154


In [31]:
org_all.shape

(178871, 16)

In [32]:
provider_all.shape

(1023054, 22)

In [54]:
provider_all[provider_all['npi']==1003062076].head().T

npi,1003062076
npi,1003062076
pac_id,4183898281
last_name,SPOJA
first_name,CHRISTOFFER
middle_name,AARON
suffix,0
gender,M
credential,0
medical_school,OTHER
graduation_year,2006


In [34]:
#DO NOT DELETE - REFERENCE DUPLICATES SPECIALITIES
#provider_all[provider_all['npi']==1073536074].head().T

In [35]:
#select WA data
wa_provider=provider_all[provider_all['state']=='WA']
wa_org=org_all[org_all['state']=='WA']

In [36]:
#save WA data
wa_provider.to_pickle('wa_provider_pickle.pkl') 
wa_org.to_pickle('wa_org_pickle.pkl') 

In [37]:
#load from pickle
wa_provider= pd.read_pickle('wa_provider_pickle.pkl')
wa_org= pd.read_pickle('wa_org_pickle.pkl')

In [38]:
wa_provider.head()

,npi,pac_id,last_name,first_name,middle_name,suffix,gender,credential,medical_school,graduation_year,...,state,related_org_ccn_1,related_org_ccn_2,related_org_ccn_3,related_org_ccn_4,related_org_ccn_5,quality_measures,electronic_measures,hearth_initiative,years_experience
npi,,,,,,,,,,,,,,,,,,,,,
1922016203,1922016203,648178822,ULLMAN,RONALD,H,0,M,MD,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,1967.0,...,WA,0.0,0.0,0.0,0.0,0.0,0,0,0,50.0
1376722587,1376722587,42391252,REARDON,THOMAS,G,0,M,0,OTHER,1989.0,...,WA,0.0,0.0,0.0,0.0,0.0,0,0,0,28.0
1942253190,1942253190,4688610686,WEAVER,MARGUERITE,0,0,F,CSW,OTHER,1994.0,...,WA,0.0,0.0,0.0,0.0,0.0,0,0,0,23.0
1285631317,1285631317,2365348307,GRAY,DOUGLAS,0,0,M,0,OTHER,1976.0,...,WA,0.0,0.0,0.0,0.0,0.0,0,0,0,41.0
1619126281,1619126281,2264593904,PARK,YOUNG,A,0,F,0,OTHER,2006.0,...,WA,0.0,0.0,0.0,0.0,0.0,0,0,0,11.0


In [39]:
#provider_all[provider_all['npi']==1790741924].head().T
#wa_provider[wa_provider.npi==1447262910]
#wa_provider[wa_provider.npi==1790741924]

In [40]:
#org_all = org_all.sort(org_all.columns[1],ascending=False)
#org_all.head(10)

In [41]:
wa_org.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3260 entries, 1376722587 to 1497702062
Data columns (total 16 columns):
prof_id              3260 non-null object
org_name             3260 non-null object
group_pac_id         3260 non-null int64
practice_size        3260 non-null float64
address              3260 non-null object
address_extra        3260 non-null object
city                 3260 non-null object
state                3260 non-null object
zip_code             3260 non-null object
phone_number         3260 non-null object
related_org_lbn_1    3260 non-null object
related_org_lbn_2    3260 non-null object
related_org_lbn_3    3260 non-null object
related_org_lbn_4    3260 non-null object
related_org_lbn_5    3260 non-null object
zip_base             3260 non-null int64
dtypes: float64(1), int64(2), object(13)
memory usage: 433.0+ KB


In [42]:
#wa_provider_x=wa_provider[wa_provider.duplicated('npi')==True]

In [43]:
#wa_provider_x.info()

In [44]:
#Build summary variables - in-work

# secondary_spec
# import pandas as pd

# def alert(c):
#   if c['used'] == 1.0:
#     return 'Full'
#   elif c['used'] == 0.0:
#     return 'Empty'
#   elif 0.0 < c['used'] < 1.0:
#     return 'Partial'
#   else:
#     return 'Undefined'

# df = pd.DataFrame(data={'portion':[1, 2, 3, 4], 'used':[1.0, 0.3, 0.0, 0.8]})

# df['alert'] = df.apply(alert, axis=1)

######combine specialities
# def split_spec(df):
#     var=set()
#     words=df['secondary_spec'].split(',')
#     var.update(words)
#     var.add(df['primary_specialty'])
#     var.add(df['specialty_1'])
#     var.add(df['specialty_2'])
#     var.add(df['specialty_3'])
#     #var.add(df['specialty_4'])
#     return var
# provider_all['years_experience'] = provider_all.apply(f, axis=1)



In [45]:
#provider_all.duplicated().value_counts()
#provider_all=provider_all.drop_duplicates()
#provider_all.drop_duplicates(*args, **kwargs)
#provider_all['npi'].value_counts()

In [46]:
#provider_wa=provider_all[provider_all['state']=='WA']
#provider_wa.info()

In [47]:
#fill all NaN with 0
#provider_wa.update(provider_wa.fillna(0))

In [48]:
#transform secondary speciality in counts
#transform hospital_affiliation in counts
#change col names

In [49]:
#provider_wa=provider_all[provider_all['state']=='WA']
#provider_wa.info()

In [50]:
#provider_wa['professional_accepts_medicare_assignment'=='M']
#x=provider_wa[provider_wa['professional_accepts_medicare_assignment']=='M']

In [51]:
#provider_wa['hospital_affiliation_lbn_5'].value_counts()